In [407]:
# Python Libraries
import random
import math
import numbers
import platform
import copy
import os
import time
import pickle
import re
import shutil

# Importing essential libraries for basic image manipulations.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import PIL

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as transforms
import torchvision.transforms.functional as tF
import torchvision.models as models

In [363]:
%matplotlib inline

# Enable/Disable GPU 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [364]:
device

device(type='cuda', index=0)

### Create custom dataset

In [365]:
class CustomImageDataset(Dataset):
    def __init__(self, img_labels, img_labels_train, img_labels_test, img_dir, transform=None, flag=False):
        
        if flag == 1:
            self.img_labels = img_labels_test
        elif flag == 0:
            self.img_labels = img_labels_train
        else:
            self.img_labels = img_labels

        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]

        
        if self.transform:
            image = self.transform(image)
            
        return image.float(), label
    

In [366]:
def split_data(annotations_file):
    img_labels = pd.read_csv(annotations_file, header = 0)
    img_labels_train, img_labels_test, _, _ = train_test_split(
                                                    img_labels,
                                                    img_labels['champion_onehot'],
                                                    test_size=0.2,
                                                    random_state = 0,
                                                    stratify=img_labels['champion_onehot'])

    print(len(np.unique(img_labels_train['champion_onehot'])))
    print(len(np.unique(img_labels_train['champion_name'])))
    print(len(np.unique(img_labels_test['champion_onehot'])))
    print(len(np.unique(img_labels_test['champion_name'])))

    
    return img_labels_train, img_labels_test
    

In [367]:
annotations_file = './data/annotation_final_file.csv'
# batch_size = 4
batch_size = 64

transform = transforms.Compose(
    [transforms.RandomVerticalFlip(p=0.5),
     transforms.RandomHorizontalFlip(p=0.5)])

img_labels_train, img_labels_test = split_data(annotations_file=annotations_file)

img_labels = pd.read_csv(annotations_file, header = 0)

trainset = CustomImageDataset(
                              img_labels = img_labels, 
                              img_labels_train = img_labels_train, 
                              img_labels_test = img_labels_test,
                              img_dir='./data/champion-classifier', transform=None, flag=0)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=0, shuffle=False)

testset = CustomImageDataset(
                              img_labels = img_labels, 
                              img_labels_train = img_labels_train, 
                              img_labels_test = img_labels_test,
                              img_dir='./data/champion-classifier', transform=None, flag=1)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=0, shuffle=False)


# trainset = CustomImageDataset(annotations_file = './data/annotation_final_file_champ_only.csv', 
#                               img_dir='./data/champion-classifier', transform=None, flag=0)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=0, shuffle=True)

# testset = CustomImageDataset(annotations_file = './data/annotation_final_file_champ_only.csv', 
#                               img_dir='./data/champion-classifier', transform=None, flag=1)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=0, shuffle=True)



85
85
78
78


### resNet18

In [368]:
class Champion_Net(nn.Module):
    def __init__(self, num_classes, criterion):
        super(Champion_Net, self).__init__()

        # Implement me
        model_ft = models.resnet18()
        model_ft.fc = nn.Linear(512, num_classes)
        
        self.model = model_ft
        self.criterion = criterion
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inp):
        
        pred = self.model(inp)
        return pred

In [369]:
def train_model(model, dataloaders, optimizer, num_epochs=25):
    
    since = time.time()
    acc_list = []
    model.train() # In training mode

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            loss = model.criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # backward + optimize only if in training phase
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Epoch information
        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        acc_list.append(epoch_acc)

        print('Training Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return acc_list

In [370]:
def eval_model(model, dataloaders):
    
    since = time.time()
    model.eval() # In training mode

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in dataloaders:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():

            # forward
            outputs = model(inputs)
            loss = model.criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    overall_loss = running_loss / len(dataloaders.dataset)
    overall_acc = running_corrects.double() / len(dataloaders.dataset)

    print('Evaluation Loss: {:.4f} Acc: {:.4f}'.format(overall_loss, overall_acc))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return overall_acc

In [371]:
def vec2Champ(vec):

    with open("data/master_champ_list.pkl", "rb") as input_file:
        master_champ_list = pickle.load(input_file)
        
    vec2Champ = {i : champ for i, champ in enumerate(master_champ_list)}
    champ = [vec2Champ.get(i) for i in vec]
    
    return champ

In [372]:
def model_pred(model, dataloaders):
    
    since = time.time()
    model.eval() # In prediction mode

    labels = []
    gt = []

    # Iterate over data.
    for inputs, groundTruth in dataloaders:
        inputs = inputs.to(device)
        
        with torch.no_grad():

            # forward
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            labels += preds.tolist()
            gt += groundTruth.tolist()


    time_elapsed = time.time() - since
    print('Prediction complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return labels, vec2Champ(labels), gt

### Training

In [373]:
champion_classifer = Champion_Net(86, nn.CrossEntropyLoss()).cuda()

In [374]:
optimizer_SGD = torch.optim.SGD(champion_classifer.parameters(), lr=0.01, momentum=0.9)

In [375]:
train_model(champion_classifer, trainloader, optimizer_SGD, 5)

Epoch 0/4
----------
Training Loss: 1.7941 Acc: 0.6850
Epoch 1/4
----------
Training Loss: 1.1851 Acc: 0.7458
Epoch 2/4
----------
Training Loss: 0.8730 Acc: 0.8042
Epoch 3/4
----------
Training Loss: 0.6000 Acc: 0.8698
Epoch 4/4
----------
Training Loss: 0.3896 Acc: 0.9275
Training complete in 0m 10s


[tensor(0.6850, device='cuda:0', dtype=torch.float64),
 tensor(0.7458, device='cuda:0', dtype=torch.float64),
 tensor(0.8042, device='cuda:0', dtype=torch.float64),
 tensor(0.8698, device='cuda:0', dtype=torch.float64),
 tensor(0.9275, device='cuda:0', dtype=torch.float64)]

### Evaluation

In [376]:
eval_model(champion_classifer, testloader)

Evaluation Loss: 0.9573 Acc: 0.7894
Training complete in 0m 0s


tensor(0.7894, device='cuda:0', dtype=torch.float64)

In [377]:
# Make prediction 
lab_test, champ_test, gt = model_pred(champion_classifer, testloader)

Prediction complete in 0m 0s


In [378]:
with open("data/master_champ_list.pkl", "rb") as input_file:
    master_champ_list = pickle.load(input_file)

gt_original = list(img_labels_test.champion_onehot)
champ_original = list(img_labels_test.champion_name)


# Calculate confision matrix + accuracy per class

# C_ij, in group i, predicted group j
cf_m = confusion_matrix(y_true = champ_original, y_pred = champ_test, labels=master_champ_list)


np.divide(np.diag(cf_m), np.sum(cf_m, axis = 1))



np.sum(np.diag(cf_m[:-1, :-1]))/np.sum(cf_m[:-1, :-1])

<ipython-input-378-2c0329542983>:14: RuntimeWarning: invalid value encountered in true_divide
  np.divide(np.diag(cf_m), np.sum(cf_m, axis = 1))


0.36574074074074076

In [379]:
# Confusion matrix test
cf_m[-1, -1]/np.sum(cf_m[-1])

0.9785330948121646

### Batch First

In [ ]:
# Batch 1
BatchSet_1 = CustomImageDataset(annotations_file = './data/batch-data-0/annotation_file-0.csv', 
                              img_dir='./data/batch-data-0', transform=None, test=2)
BatchSetLoader_1 = torch.utils.data.DataLoader(BatchSet_1, batch_size=10, num_workers=0, shuffle=True)

In [ ]:
df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'batch-data-0', 'annotation_file-0.csv'), index_col=False)

df.champion_name = champ_1
df.champion_onehot = lab_1

df.to_csv(os.path.join(os.getcwd(), 'data', 'batch-data-0', 'annotation_file-0.csv'), index = False)

### Save / Load

In [ ]:
torch.save(champion_classifer.state_dict(), './model/champion_classifier_bin0.pt')

In [ ]:
champion_classifer = Champion_Net(86, nn.CrossEntropyLoss()).to(device)
champion_classifer.load_state_dict(torch.load('./model/champion_classifier_bin0.pt', 
                                              map_location=device))

### Predict currently trained model on master data sheet

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, img_labels, img_dir):
        

        self.img_labels = img_labels
        self.img_dir = img_dir

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]

        return image.float()

In [ ]:
master_data_path = os.path.join(os.getcwd(), 'data', 'Champions_Label.xlsx')
master_data = pd.read_excel(master_data_path)

In [404]:
master_data_sample = master_data[0:10]

master_data_sample

,Unnamed: 0,youtuber,video_name,frame_name,cropped_name,champion_name,predicted,image_name,image_file_path
0,0,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_0.jpg,NaN,NaN,image_0.jpg,/Users/mgw/Desktop/CS686_Project/teamfight-tac...
1,1,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_1.jpg,Warwick,NaN,NaN,NaN
2,2,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_2.jpg,Taric,NaN,NaN,NaN
3,3,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_3.jpg,Camille,NaN,NaN,NaN
4,4,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_4.jpg,Background,NaN,NaN,NaN
5,5,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_5.jpg,Background,NaN,NaN,NaN
6,6,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_6.jpg,Background,NaN,NaN,NaN
7,7,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_7.jpg,NaN,NaN,image_1.jpg,/Users/mgw/Desktop/CS686_Project/teamfight-tac...
8,8,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_8.jpg,NaN,NaN,image_2.jpg,/Users/mgw/Desktop/CS686_Project/teamfight-tac...
9,9,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_9.jpg,NaN,NaN,image_3.jpg,/Users/mgw/Desktop/CS686_Project/teamfight-tac...


In [412]:
master_img_input_path = os.path.join(os.getcwd(), 'data', 'all_img')

if not os.path.exists(master_img_input_path):
    os.makedirs(master_img_input_path)


# Create master_annotation file

# for i in range(len(master_data_sample)):
#     youtuber = master_data_sample.iloc[i,:].youtuber
#     video_name = master_data_sample.iloc[i,:].video_name
#     frame_name = master_data_sample.iloc[i,:].frame_name
#     frame_name = re.sub(r'.jpg$', '', frame_name)
    
#     cropped_name = master_data_sample.iloc[i,:].cropped_name
#     cropped_name = re.sub(r'.jpg$', '.png', cropped_name)
    
#     to_img_dir = os.path.join(os.getcwd(), 'contents', youtuber, video_name, frame_name, cropped_name)
#     final_img_dir = os.path.join(master_img_input_path, f'image-{i}.jpg')
    
#     shutil.copyfile(to_img_dir, final_img_dir)
    
    


FileNotFoundError: [Errno 2] No such file or directory: 'F:\\PycharmProjects\\TFT\\contents\\Mortdog\\100 HP Challenger - Nerfed but still so good  TFT Gizmos _ Gadgets  Teamfight Tactics.mp4\\frame11880\\cropped_S_0.png'

In [409]:
testset = CustomImageDataset(annotations_file = annotations_file, 
                          img_labels = img_labels, 
                          img_labels_train = None, 
                          img_labels_test = None,
                          img_dir='./data/champion-classifier', transform=None, flag=None)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=0, shuffle=False)

,Unnamed: 0,youtuber,video_name,frame_name,cropped_name,champion_name,predicted,image_name,image_file_path
0,0,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_0.jpg,NaN,NaN,image_0.jpg,/Users/mgw/Desktop/CS686_Project/teamfight-tac...
1,1,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_1.jpg,Warwick,NaN,NaN,NaN
2,2,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_2.jpg,Taric,NaN,NaN,NaN
3,3,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_3.jpg,Camille,NaN,NaN,NaN
4,4,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_4.jpg,Background,NaN,NaN,NaN
5,5,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_5.jpg,Background,NaN,NaN,NaN
6,6,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_6.jpg,Background,NaN,NaN,NaN
7,7,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_7.jpg,NaN,NaN,image_1.jpg,/Users/mgw/Desktop/CS686_Project/teamfight-tac...
8,8,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_8.jpg,NaN,NaN,image_2.jpg,/Users/mgw/Desktop/CS686_Project/teamfight-tac...
9,9,Mortdog,100 HP Challenger - Nerfed but still so good ...,frame11880.jpg,cropped_S_9.jpg,NaN,NaN,image_3.jpg,/Users/mgw/Desktop/CS686_Project/teamfight-tac...
